In [1]:
#conda install -c conda-forge geopandas

In [2]:
#conda update -n base -c defaults conda

In [3]:
import numpy as np
import pandas as pd

In [4]:
dong = pd.read_excel("geojson_행정동코드.xlsx") ## 누락된 행정동 이름 비교 후 입력
dis = pd.read_excel("연금.xlsx")

In [5]:
dong

,시,구,동,행정동코드
0,서울특별시,종로구,사직동,11110530
1,서울특별시,종로구,삼청동,11110540
2,서울특별시,종로구,부암동,11110550
3,서울특별시,종로구,평창동,11110560
4,서울특별시,종로구,무악동,11110570
...,...,...,...,...
419,서울특별시,강동구,둔촌1동,11740690
420,서울특별시,강동구,둔촌2동,11740700
421,서울특별시,강동구,암사1동,11740570
422,서울특별시,강동구,천호2동,11740610


In [6]:
dis

,Unnamed: 0,구,행정동별연금평균
0,4,종로구,4725.843364
1,5,중구,4533.399237
2,6,용산구,5136.380812
3,7,성동구,4836.889273
4,8,광진구,4887.699283
5,9,동대문구,4303.514117
6,10,중랑구,4258.907756
7,11,성북구,4504.434598
8,12,강북구,4068.676272
9,13,도봉구,4713.611745


In [7]:
# dis "구" 기준으로 정렬
dis.sort_values(by="구",inplace=True)

In [8]:
# dis index 다시 설정
dis.reset_index(inplace=True)

In [9]:
# 필요없는 컬럼 삭제
dong.drop(["시","행정동코드"], axis=1, inplace=True)

In [10]:
dong_no = dong.groupby("구").count()
dong_no.sort_values(by="구",inplace=True)
dong_no.reset_index(inplace=True)
dong_no

,구,동
0,강남구,22
1,강동구,18
2,강북구,13
3,강서구,20
4,관악구,21
5,광진구,15
6,구로구,15
7,금천구,10
8,노원구,19
9,도봉구,14


In [11]:
dis["동"] = dong_no["동"]

In [12]:
# 동별장기요양병원 = 정원 / 동
dis["행정동별연금평균"] = dis["행정동별연금평균"] / dis["동"]

In [13]:
dong.sort_values("구")

,구,동
362,강남구,대치1동
375,강남구,압구정동
376,강남구,청담동
377,강남구,대치2동
378,강남구,개포2동
...,...,...
105,중랑구,신내1동
109,중랑구,망우본동
108,중랑구,면목3.8동
101,중랑구,중화2동


In [14]:
dong.sort_values(["구","동"], inplace=True)

In [15]:
# 각 구의 개수를 알고있기 때문에 그 수만큼 동* "동별장기요양병원" 대입
# 예를들어 강남구가 5개이고 "동별장기요양병원" 이 29.318182 일 경우
# 10 * [29.318182] 를 하면 [29.318182, 29.318182, 29.318182, 29.318182, 29.318182, 29.318182] 가 된다.
# 이를 다더하면 2차원 배열이 된다. 매 for 문 마다 배열이 dong_val 배열에 들어가기 때문에
final = []
for i in range(len(dis)):
    final += (dis.iloc[i]["동"] * [dis.iloc[i]["행정동별연금평균"]])

In [16]:
# 위에서 인덱스를 조정했기 때문에 인덱스를 다시 설정. 설정하지 않을 경우 데이터를 추가할 시 이상하게 추가된다
dong.reset_index(inplace=True)
dong

,index,구,동
0,368,강남구,개포1동
1,378,강남구,개포2동
2,369,강남구,개포4동
3,358,강남구,논현1동
4,359,강남구,논현2동
...,...,...,...
419,99,중랑구,상봉2동
420,105,중랑구,신내1동
421,106,중랑구,신내2동
422,100,중랑구,중화1동


In [17]:
t = pd.Series(final)
dong["행정동별연금평균"] = pd.Series(t)

In [18]:
dong

,index,구,동,행정동별연금평균
0,368,강남구,개포1동,306.970009
1,378,강남구,개포2동,306.970009
2,369,강남구,개포4동,306.970009
3,358,강남구,논현1동,306.970009
4,359,강남구,논현2동,306.970009
...,...,...,...,...
419,99,중랑구,상봉2동,266.181735
420,105,중랑구,신내1동,266.181735
421,106,중랑구,신내2동,266.181735
422,100,중랑구,중화1동,266.181735


In [19]:
# 필요없는 컬럼지우기
dong.drop("index", axis=1, inplace=True)

In [20]:
nums = pd.read_excel("geojson_행정동코드.xlsx") ## 행정동 코드 추가

In [21]:
nums

,시,구,동,행정동코드
0,서울특별시,종로구,사직동,11110530
1,서울특별시,종로구,삼청동,11110540
2,서울특별시,종로구,부암동,11110550
3,서울특별시,종로구,평창동,11110560
4,서울특별시,종로구,무악동,11110570
...,...,...,...,...
419,서울특별시,강동구,둔촌1동,11740690
420,서울특별시,강동구,둔촌2동,11740700
421,서울특별시,강동구,암사1동,11740570
422,서울특별시,강동구,천호2동,11740610


In [22]:
# 필요없는 컬럼 삭제
nums.drop(["시"], axis=1, inplace=True)

In [23]:
all_df = pd.merge(left=dong, right=nums,how='left',
                  on=['구','동'], sort=False)

In [24]:
all_df

,구,동,행정동별연금평균,행정동코드
0,강남구,개포1동,306.970009,11680660
1,강남구,개포2동,306.970009,11680670
2,강남구,개포4동,306.970009,11680690
3,강남구,논현1동,306.970009,11680521
4,강남구,논현2동,306.970009,11680531
...,...,...,...,...
419,중랑구,상봉2동,266.181735,11260590
420,중랑구,신내1동,266.181735,11260680
421,중랑구,신내2동,266.181735,11260690
422,중랑구,중화1동,266.181735,11260600


In [25]:
all_df = all_df[['행정동코드', '구', '동', '행정동별연금평균']]

In [26]:
all_df

,행정동코드,구,동,행정동별연금평균
0,11680660,강남구,개포1동,306.970009
1,11680670,강남구,개포2동,306.970009
2,11680690,강남구,개포4동,306.970009
3,11680521,강남구,논현1동,306.970009
4,11680531,강남구,논현2동,306.970009
...,...,...,...,...
419,11260590,중랑구,상봉2동,266.181735
420,11260680,중랑구,신내1동,266.181735
421,11260690,중랑구,신내2동,266.181735
422,11260600,중랑구,중화1동,266.181735


In [27]:
# all_df.to_excel("행정동별로_나눈_연금.xlsx")

In [28]:
import sklearn
sklearn.__version__

import pandas as pd
import numpy as np
import folium

from sklearn.preprocessing import MinMaxScaler

In [29]:
scaler= MinMaxScaler()

all_df[["행정동별연금평균"]]= scaler.fit_transform(all_df[["행정동별연금평균"]])

all_df

,행정동코드,구,동,행정동별연금평균
0,11680660,강남구,개포1동,0.385853
1,11680670,강남구,개포2동,0.385853
2,11680690,강남구,개포4동,0.385853
3,11680521,강남구,논현1동,0.385853
4,11680531,강남구,논현2동,0.385853
...,...,...,...,...
419,11260590,중랑구,상봉2동,0.215713
420,11260680,중랑구,신내1동,0.215713
421,11260690,중랑구,신내2동,0.215713
422,11260600,중랑구,중화1동,0.215713


In [30]:
all_df.to_excel("연금_정규화.xlsx")

In [31]:
low = all_df.sort_values(by = ['행정동별연금평균', '행정동코드'], ascending = True).head(10)
#low

In [32]:
high = all_df.sort_values(by = ['행정동별연금평균', '행정동코드'], ascending = False).head(10)
#high

In [33]:
low = low[['행정동코드', '구', '동']]
low

,행정동코드,구,동
79,11620525,관악구,보라매동
92,11620545,관악구,청림동
83,11620565,관악구,성현동
93,11620575,관악구,행운동
73,11620585,관악구,낙성대동
91,11620595,관악구,청룡동
87,11620605,관악구,은천동
90,11620615,관악구,중앙동
88,11620625,관악구,인헌동
76,11620630,관악구,남현동


In [34]:
high = high[['행정동코드', '구', '동']]
high

,행정동코드,구,동
133,11545710,금천구,시흥5동
132,11545700,금천구,시흥4동
131,11545690,금천구,시흥3동
130,11545680,금천구,시흥2동
129,11545670,금천구,시흥1동
128,11545640,금천구,독산4동
127,11545630,금천구,독산3동
126,11545620,금천구,독산2동
125,11545610,금천구,독산1동
124,11545510,금천구,가산동
